# Clustering

#### Internet Analytics - Lab 3

**Group**: K

**Names**:

- Mathieu Sauser
- Luca Mouchel
- Heikel Jebali
- Jérémy Chavot

### Exercise 3.12

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.cluster import KMeans

from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, SaveTool, ResetTool, PanTool, BoxZoomTool, WheelZoomTool, UndoTool

from itertools import combinations

%matplotlib inline
plt.style.use("ggplot")

/opt/anaconda3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [2]:
tagNewCoordinates = np.load('./tagNewCoordinates.npy', allow_pickle=True).item()
data = [(list(values)) for key, values in list(tagNewCoordinates.items())]

In [4]:
def plotData(kMeans, dim1, dim2):
    name = HoverTool(tooltips=[('Name:','@name'),])
    tools = [name, SaveTool(), ResetTool(), PanTool(), BoxZoomTool(), WheelZoomTool(), UndoTool()]
    plot = figure(plot_width=800, plot_height=400, tools=tools, title=f'Principle directions {dim1} and {dim2}.')
    
    clusters = kMeans.cluster_centers_
    colors = ["red", "green", "orange", "yellow", "purple"]
    
    dataSource = ColumnDataSource(
        data = {
            'dim1': [x[dim1] for x in data],
            'dim2': [x[dim2] for x in data],
            'name': list(tagNewCoordinates.keys()),
            'color': [colors[cluster] for cluster in kMeans.labels_]
        }
    )
    plot.circle('dim1','dim2', source=dataSource, size=5, alpha=0.4, color='color')
    
    centersSource = ColumnDataSource(
        data = {
            'dim1': [x[dim1] for x in clusters],
            'dim2': [x[dim2] for x in clusters],
            'name': ['cluster_center' for i in range(5)]
        }
    )
    plot.triangle('dim1', 'dim2', source=centersSource, size=10, alpha=0.6, color='black')

    show(plot, notebook_handle=True)

In [5]:
dims = combinations(range(5), 2)

kMeans = KMeans(n_clusters=5).fit(data)
for dim in dims:
    plot = plotData(kMeans, dim[0], dim[1])
    output_notebook()

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

### Exercise 3.13